<h1>Setup

In [1]:
import pandas as pd
import numpy as np
from random import randrange

# import matplotlib.pyplot as plt
# import seaborn as sns
# %matplotlib inline
pd.set_option('display.max_columns', 500)

In [2]:
filename = 'cleaned_fraudTrain.csv'
# import Dataset
path = '../dataset/midterm_dataset/'+filename
df = pd.read_csv(path)

row_num = df.shape[0]
feature_num = df.shape[1]

nor_col = ['amt', 'city_pop','lat', 'long', 'unix_time', 'age']
        
df[nor_col] = df[nor_col].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

label = 'is_fraud'

features = df.loc[:, df.columns != label].columns.tolist()

df_ig = pd.read_csv('info_gain_all_feature.csv')
df_ig = df_ig.rename(columns={'Unnamed: 0' : 'features'})
df_ig.set_index("features", inplace = True)

In [3]:
# import Dataset
path = '../dataset/midterm_dataset/cleaned_fraudTest.csv'
df_test = pd.read_csv(path)

row_num = df_test.shape[0]
feature_num = df_test.shape[1]

nor_col = ['amt', 'city_pop','lat', 'long', 'unix_time', 'age']
        
df_test[nor_col] = df_test[nor_col].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

X_test = df_test[features]

y_test = df_test[label]

<h1>Function

In [4]:
def find_prob_each(label, feature, df=df):
    """
    -helper function for group_entropy function
    -and also give probability often its occures compare to the label
    
    return dict of probability of feature
    """
    if feature not in df.columns.tolist():
        return {'never_met': [0, 0]}

    temp_df = df.groupby([feature, label])
    prob_list = []

    if len(temp_df.size())< 2000 and feature not in ['age', 'lat', 'long', 'city_pop']:  
        for r in range(len(temp_df.size())):
                feature_prob = temp_df.size().values[r]/ \
                    len(df[feature][df[feature] == temp_df.size().index[r][0]])
                prob_feature = {}
                prob_feature[temp_df.size().index[r]] = feature_prob
                featu = ''
                if len(prob_list) == 0:
                    featu = temp_df.size().index[r][0]
                    prob_list.append({featu : [0.0, 0.0]})
                    if temp_df.size().index[r][1] == 1:
                        prob_list[-1][featu][1] = feature_prob
                    else:
                        prob_list[-1][featu][0] = feature_prob
                elif temp_df.size().index[r][0] in prob_list[-1]: #append 2nd value
#                     print(df.groupby([feature, label]).size().index[r][0], r,feature_prob, temp_df.size().index[r][1])
#                     print(prob_list[-1][temp_df.size().index[r][0]])
                    if temp_df.size().index[r][1] == 1:
                        prob_list[-1][temp_df.size().index[r][0]][1] = feature_prob
                    else:
                        prob_list[-1][temp_df.size().index[r][0]][0] = feature_prob
                else:
                    featu = temp_df.size().index[r][0]
                    prob_list.append({featu : [0.0, 0.0]})
                    if temp_df.size().index[r][1] == 1:
                        prob_list[-1][featu][1] = feature_prob
                    else:
                        prob_list[-1][featu][0] = feature_prob
                    
    else: #numeric feature
        greater_than_eq_df = df[[label, feature]][df[feature] >=df[feature].mean()]
        less_than_df = df[[label, feature]][df[feature] < df[feature].mean()]
        condition_dfs = [greater_than_eq_df, less_than_df]
    
        prob_list = [{'greater_than_eq_mean': []}, {'less_than_mean': []}]
    
        for d in condition_dfs:
            for r in range(len(d[label].value_counts())):
                feature_prob = d[label].value_counts()[r]/len(d)
#                 print(feature_prob)
                if  d.equals(greater_than_eq_df):
                    prob_list[0]['greater_than_eq_mean'].append(feature_prob)

                else:
                    prob_list[1]['less_than_mean'].append(feature_prob)
    p_dict = {}
    [p_dict.update(i) for i in prob_list]
    
    return p_dict


store_prob_dict = {}
def extract_feature_point(feature, value, df=df, store_prob_dict=store_prob_dict):

    if feature not in df.columns.tolist():
        return [0,0]
    #check type
    if type(value) != str and feature not in ['is_male', 'trans_year', 'trans_month']:
        mean_val = df[feature].mean()
        if value >= mean_val:
            value = 'greater_than_eq_mean'
        else:
            value = 'less_than_mean'
    
    print(feature, 'value', value)
        
    if feature not in store_prob_dict:
        prob_dict = find_prob_each(label, feature, df)
        store_prob_dict.update({feature : prob_dict})
        print(feature)
        print('store_dict length:', len(store_prob_dict))
        
    
    #append value
    if value not in store_prob_dict[feature]:
        #return 0 when hv no data
        return [0, 0]
    elif value in store_prob_dict[feature]:
        # print(store_prob_dict[feature])
        #return point
        return [
            100*store_prob_dict[feature][value][0]*float(df_ig.loc[feature,:]),
            100*store_prob_dict[feature][value][1]*float(df_ig.loc[feature,:])
        ]

def row_desicion_tree_predictor(an_row=[], features=[], df=df, store_prob_dict=store_prob_dict, x_features=features):
    not_fraud = []
    fraud = []
#     an_row = an_df_row.values.tolist()[0]
    #making index of sub sample and test data to be the same
    #in case x_features lenght is more than df features
    for i, feature in enumerate(x_features):
        point = extract_feature_point(feature, an_row[i], df, store_prob_dict)
        not_fraud.append(point[0])
        fraud.append(point[1])
#         print(feature,':',point,'done')
    print('predicting..')
#     print(1)
    #predicting
    nfra_mean = np.mean(not_fraud)
    fra_mean = np.mean(fraud)
    if nfra_mean >= fra_mean:
        return 0
    else:
        return 1

    
def desicion_tree_predictor(X, train, features, store_prob_dict={}):
    """
    return list of predicting point
    """
    x_features = X.loc[:, X.columns != label].columns.tolist()
    x_list = X.values.tolist()
    return [row_desicion_tree_predictor(row, features, train, store_prob_dict, x_features) for row in x_list]

<h1>call storing var

In [5]:
%store -r store_prob_dict_list
%store -r subset_features_per_df_list

In [6]:
store_prob_dict_list[1]

{}

In [7]:
subset_features_per_df_list[1]

['amt', 'trans_time', 'job', 'unix_time', 'is_male', 'long', 'is_fraud']

In [8]:
df_node = df[subset_features_per_df_list[1]]
df_node.head()

,amt,trans_time,job,unix_time,is_male,long,is_fraud
0,0.000137,Late Night,"Psychologist, counselling",0.000000e+00,0,0.864638,0
1,0.003670,Late Night,Special educational needs teacher,5.598526e-07,0,0.485682,0
2,0.007569,Late Night,Nature conservation officer,7.105821e-07,1,0.546553,0
3,0.001520,Late Night,Patent attorney,1.248902e-06,1,0.548070,0
4,0.001415,Late Night,Dance movement psychotherapist,3.617509e-06,1,0.882190,0


In [9]:

y_pred_node2 = desicion_tree_predictor(
            X=X_test,
            train=df_node,
            features=df_node.loc[:, df_node.columns != label].columns.tolist(),
            store_prob_dict=store_prob_dict_list[1]
        )

ale value 1
long value greater_than_eq_mean
job value Soil scientist
unix_time value greater_than_eq_mean
trans_time value Night
predicting..
amt value greater_than_eq_mean
is_male value 0
long value greater_than_eq_mean
job value Call centre manager
unix_time value greater_than_eq_mean
trans_time value Night
predicting..
amt value less_than_mean
is_male value 1
long value less_than_mean
job value Administrator, education
unix_time value greater_than_eq_mean
trans_time value Night
predicting..
amt value less_than_mean
is_male value 0
long value greater_than_eq_mean
job value Make
unix_time value greater_than_eq_mean
trans_time value Night
predicting..
amt value greater_than_eq_mean
is_male value 1
long value less_than_mean
job value Clothing/textile technologist
unix_time value greater_than_eq_mean
trans_time value Night
predicting..
amt value less_than_mean
is_male value 0
long value greater_than_eq_mean
job value Senior tax professional/tax inspector
unix_time value greater_than_eq_m

In [10]:
print(len(y_pred_node2))
print('done')

555719
done


In [11]:
%store y_pred_node2

Stored 'y_pred_node2' (list)


In [12]:
import pickle

rdf_classifier_node2 = open('random_forest_y_pred_node/rdf_classifier_node2.pkl', 'wb')

pickle.dump(y_pred_node2, rdf_classifier_node2)

rdf_classifier_node2.close()
print('closed')

closed
